# Classification of Unstructured Documents 
## *Transfer Learning with BERT*
### GRAD-E1394 Deep Learning

---

Authors:
*   Ma. Adelle Gia Arbo, m.arbo@students.hertie-school.org
*   Janine De Vera, j.devera@students.hertie-school.org
*   Lorenzo Gini, l.gini@students.hertie-school.org
*   Lukas Warode, l.warode@students.hertie-school.org | lukas.warode@gmx.de

---

This tutorial demonstrates the pipeline for classifying unstructured documents, particularly those in PDF format. 


# Table of Contents


*   [Memo](#memo)
*   [Overview](#overview)
*   [Background & Prerequisites](#background-and-prereqs)
*   [Software Requirements](#software-requirements)
*   [Data Description](#data-description)
*   [Methodology](#methodology)
*   [Results & Discussion](#results-and-discussion)
*   [References](#references)


<a name="memo"></a>
# Memo

Write a memo for the leadership explaining in layman's terms why this topic is relevant for public policy. Discuss relevant research works, real-world examples of successful applications, and organizations and governments that apply such approaches for policy making. 


### *Sketch Lukas (not done)*

#### Main Point / Take Away (Executive Summary???)

#### Background 

Companies as well as public and political institutions are able and ofent required to access loads of information nowadays. In doing that, they often face the problem of unstructured information, especially when dealing with text data. Data in textual form is the most common type of unstructured, unfortunately, it is also representing the most fundamental type of documents with respect to policymakers and public institutions: Legal documents, bills and policy papers are just some examples of common text sources, which are part of the daily business in the political world. Unstructured text data entails a variety of different problems when it comes actually gaining quantitative analytical insights from them. Computers commonly have difficulties understanding textual data. Analytical and technical competences are also scarce: Only 18% of companies are able to use unstructured data, while most organizations are make their (data-driven) decisions on the basis of only 10 to 20% of their available data source. The situation for public institutions is worsend by the fact that most of the modern text data analysis frameworks and models are dominated by different industry players, while being designed and trained according to the industry-specific needs, which do not transfer to the needs of the mentioned political text sources. The EU Commission (or directly mention DMA?) …

This motivates the necessity of implementing …

#### Evidence (Sktech our technical solution?)
- Text Classification Pipepline + BERT Model(ling)

- Results??


#### Conclusion and Implementation

- Results??

- Being specific about implementation and actual usage (maybe even specify DMA context a bit more and say who and what stakeholders/institutions are taking what position in context)

*Open points*

- Include practicality and gained efficiency
- Normative perspective: "Technical" objectivity (but one needs to be careful for potential biases)

\
- Directly mention DMA at beginning?
- How structured should the memo be?

\
- Sources need to be added for numbers of slides (e.g. 18% of companies blabla)


*Structure*
![](https://mitcommlab.mit.edu/broad/wp-content/uploads/sites/5/2017/12/policy-memo-struct-700x843.png)

![](https://www.bu.edu/sph/files/2014/06/Policy-Memo-Organization.jpg)

<a name="overview"></a>
# Overview

Over 80% of all data is *unstructured*. Most of the information we consume come in a format that is not organized in a pre-defined manner (e.g. tables) or with a specific data model in mind (e.g. matrices). 

<u>Text</u> is the most common type of unstructured data and it comes in a variety of forms like blogs, news articles, social media content, as well as official documents. This lack of structure that can be readily understood by machines is what makes it difficult to maximize text as a data source. Algorithms that efficiently and accurately process text would have a variety of applications in organisations, especially public institutions that have access to different types of documents. 

In this tutorial we demonstrate one such application in the context of the European Commission (EC). Whenever new legislation is proposed, the EC opens **public consultations** where various stakeholders (e.g. businesses, academia, law firms, associations, private individuals) submit documents that detail their views on the proposal. The EC receives anywhere between 10,000 to 4 million of these public consultation documents annually. Using machine learning and deep learning methods to process these documents would streamline the Commission's review of stakeholder comments, which will consequently allow them to integrate more information into their policymaking process. 

The main goal of this tutorial is to walk you through the steps of building a <u>document classifier</u> using a deep learning model called **Bidirectional Encoder Representations from Transformers (BERT)**. By the end of this tutorial you will understand how to:  

> 1. Extract, clean, and pre-process information from PDF documents
> 2. Use the pre-processed text as input to machine learning/deep learning models
> 3. Build a text/document classifier with BERT 
> 4. Compare BERT with text classifiers built using other models

We will then apply these learnings to accomplish a research objective: 

 > Classify public consultation documents of the recently enacted **Digital Markets Act** according to whether a stakeholder **agrees or disagrees** with the DMA proposal. 

The Digital Markets Act is a regulation in the European Union which came into force this 2022. It aims to promote fair competition within the digital market by defining rules for “gatekeepers” or large online platforms. Majority of the public consultation documents submitted for the DMA are from companies and business associations who will likely be affected by the law.

<a name="background-and-prereqs"></a>
# Background & Prerequisites

For this tutorial, you would need to be familiar with object oriented programming, common python libraries such as *numpy* and *pandas*, and libraries used for model building, such as *scikit-learn* and *pytorch*. Working knowledge of common language processing concepts (e.g. stemming, lemmatization, TF-IDF, embeddings) would also be useful. 

Important concepts that need to be explained briefly:  

## Reading materials

For detailed explanations of the topics covered in this tutorial, below is a list helpful references.

**Basics of natural language processing models:**
* Vajjala, S., Majumder, B., Gupta, A., & Surana, H. (2020). Practical natural language processing: a comprehensive guide to building real-world NLP systems. O'Reilly Media.


**BERT:**
* Horev, R. (2018). BERT Explained: State of the art language model for NLP. Towards Data Science, 10.
* 


<a name="software-requirements"></a>
# Software Requirements
To install software requirements and dependencies, please create a new environment using the *environment.yml* file which accompanies this notebook. 

In [ ]:
!conda env create -f environment.yml

In [81]:
# Data visualization
import matplotlib.pyplot as plt 

# Data manipulation
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile

In [82]:
# Parsing and pre-processing
from glob import glob
import os 
import re

from pdfminer.high_level import extract_text
from langdetect import detect, DetectorFactory

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [83]:
# Vector representations and embeddings
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

In [84]:
# Logistic and XGboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score
from xgboost import XGBClassifier

In [85]:
# LSTM 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import gc

/Users/janinedevera/opt/miniconda3/envs/watermelon/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [86]:
# BERT models
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler
import transformers
from transformers import AutoModel, BertTokenizerFast
from tqdm.notebook import tqdm

In [87]:
# specify GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

<a name="data-description"></a>
# Data Description

As mentioned in the [Overview](#overview), the methods discussed in this tutorial will be applied to public consultation documents of the **Digital Markets Act (DMA)**. Respondents answered a survey where they were asked a series of questions about the proposed law. The public consultation received **188 survey responses**. Some respondents also provided **accompanying position papers and reports** where they discuss their views in detail. 

To build our document classifier we need the raw **text** from the public consultation submissions, and a corresponding **label** for each document which indicates whether or not the author/s agree or disagree with the DMA proposal. 

**Text:**
The submissions come in the form of **pdf files**. These documents need to be parsed in order to extract raw text. 

**Labels:**
The labels are extracted from the **survey**, specifically from the question: 

> *"Do you consider that there is a need for the Commission to be able to intervene in gatekeeper scenarios to prevent/address structural competition problems?"*
 

## Data Download

The pdf documents and the survey used to generate labels can be downloaded from the <a href="https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12416-New-competition-tool/public-consultation_en">DMA consultation page</a>. 

## Data Cleaning

### A. Text

First, create a list of file paths of all pdf submissions: `pdf_list`. 

Each of the downloaded public consultation documents have a unique alpha-numeric ID. For example: the document ID for **"F549293-Statement_on_the_New_Competition_Tool"** is *F549293*. A regular expression is used to extract this from the file paths. The IDs are then saved in a list called `pdf_id`. 

In [9]:
pdf_dir = "data/reports/"
pdf_list = glob(os.path.join(pdf_dir, "*.pdf"))
len(pdf_list)

97

In [10]:
pdf_id = [re.search('[F][0-9]{6}', i)[0] for i in pdf_list]
pdf_id = list(set(pdf_id))
len(pdf_id) # no. of unique pdf submission

89

### B. Labels

The Excel file containing survey answers and details about the respondents is imported as a pandas dataframe. The survey dataset also has a reference column which corresponds to the reference numbers in `pdf_id`.

In [11]:
dma = pd.read_excel("data/DMA Contributions_Clean.xlsx", sheet_name='Results')
dma.head()

,Reference,Feedback date,Language,User type,First name,Surname,Scope,Organisation name,Transparency register number,Organisation size,...,Q197,Q198,Q199,Q200,Q201,Q202,Q203,Q204,Q205,Q206
0,F550828,8.09.2020 23:57,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,See our response above.,Somewhat effective,Very effective,Very effective,Most effective,Very effective,See our response above.,NaN,No.,Yes
1,F550827,8.09.2020 23:54,English,NGO (Non-governmental organisation),Juliane,von Reppert-Bismarck,NaN,Lie Detectors,094738529674-10,Small (< 50 employees),...,NaN,Not effective,Most effective,Somewhat effective,Most effective,Most effective,An additional regulatory framework imposing ob...,Lie_Detectors_-_Digital_Services_Act_-_New_Com...,Please see attached submission,Yes
2,F550826,8.09.2020 23:50,German,Business Association,Antje,Woltermann,NaN,Zentralverband Deutsches Kfz-Gewerbe e.V.,71649103246-10,Medium (< 250 employees),...,NaN,Somewhat effective,Most effective,Somewhat effective,Sufficiently effective,Very effective,-,Positionspapier_Gleichberechtigter_Zugang_zum_...,-,Yes
3,F550825,8.09.2020 23:45,English,Academic/Research Institution,DIANA,MONTENEGRO,NaN,Master student candidate Master in Competition...,NaN,Large (250 or more),...,The new procedural must be agile.,Not effective,Very effective,Very effective,Very effective,Very effective,all are adequated,PREPARATIVE_DOCUMENT_FOR_THE_EUROEPAN_COMMISIO...,The document I prepared was more than 1MB.,Yes
4,F550824,8.09.2020 23:33,English,Business Association,Kamila,Sotomska,NaN,Zwi?zek Przedsi?biorców i Pracodawców,868073924175-77,Small (< 50 employees),...,NaN,Not applicable /No relevant experience or know...,Not applicable /No relevant experience or know...,Not applicable /No relevant experience or know...,Not applicable /No relevant experience or know...,Not applicable /No relevant experience or know...,NaN,NaN,NaN,Yes


As we saw above, there are only 89 unique PDF submissions. We now check how many of survey respondents submitted supplementary papers and/or reports. An indicator variable is added to the survey dataframe to specify whether a response has an equivalent PDF submission. Note that we can only use observations that have both the document submission and survey response. 

In [12]:
# count unique Reference ids with pdf submission/s
len(dma.query('Reference in @pdf_id'))

85

In [13]:
# tag identifier with pdf submission = 1
dma['submit'] = [1 if v in pdf_id else 0 for v in dma['Reference'].values]
dma['submit'].value_counts()

0    95
1    85
Name: submit, dtype: int64

The question that will be used as a label is Q132. 
*"Do you consider that there is a need for the Commission to be able to intervene in gatekeeper scenarios to prevent/address structural competition problems?"*

We check how many Yes and No responses there are. Respondents who answered not applicable can be considered as **not in agreement** with the proposal. Their answers are lumped together with the No responses.

In [14]:
# Q132 as labels
dma.Q132.value_counts()

Yes                                                    115
Not applicable /no relevant experience or knowledge     46
No                                                      19
Name: Q132, dtype: int64

In [32]:
dma['label_132'] = np.where(dma['Q132'] == 'Not applicable /no relevant experience or knowledge', 'No', dma['Q132'])
dma.label_132.value_counts()

Yes    115
No      65
Name: label_132, dtype: int64

<a name="methodology"></a>
# Methodology

This section of the tutorial is a step-by-step walkthrough of text classification pipeline, using DMA public consultation documents as described above. Below is an outline of this section: 

<ol type="A">
  <li>Data Preparation </li>
  <li>Text Representation</li>
  <li>Model Training (with hyperparameter tuning)</li>
  <ol>
    <li> Training baseline models (logistic, XGBoost)
    <li> Training a DL classifier (LSTM)
    <li> Transfer learning with BERT and other variants
  </ol>
  <li> Model Evaluation
</ol>

## A. Data Preparation 
In this section of the tutorial, we will (1) parse PDF submissions from the Digital Markets Act public consultation and (2) pre-process raw text to keep only the most relevant information. 

### A.1 Parsing PDFs

PDF (Portable Document Format) documents are hard to work with because this format was not designed as a data input. Instead, the PDF contains a set of instructions that describe how characters or objects are positioned on a page. 

Python has text analytics libraries that convert PDFs into the required encoding format. There are several of these PDF libraries, however, they sometimes yield varying results. Here, we demonstrate one such library, `pdfminer`. We have also tried other libraries like `pdfplumber`, so feel free to also experiment on which PDF parser works best for your corpus. 

We create a dataframe `df_text` which contains information for each document - the reference number (unique ID), file name, and complete text of the document. The full text of the document is parsed using the PDF library.

In [16]:
df_text = pd.DataFrame(columns = ['Reference', 'file_name', 'text'])

for pdf_file in pdf_list:
    Reference = re.search('[F][0-9]{6}', pdf_file)[0]
    file_name = re.search('[F][0-9]{6}(.*)[\\>.]', pdf_file)[0]
    text = extract_text(pdf_file)
    row = pd.DataFrame({'Reference': Reference,'file_name': file_name, 'text': text}, index=[0])
    df_text = pd.concat([row,df_text.loc[:]]).reset_index(drop=True)

df_text.head()

,Reference,file_name,text
0,F550241,F550241-190326-Dobson_report-FINAL_VERSION.,Ref. Ares(2020)4669723 - 08/09/2020\n\nLEVELLI...
1,F550737,F550737-Mediaset_-_NCT_-_Position_paper_-_fina...,New Competition Tool \n\nPosition Paper \n\nMe...
2,F550604,F550604-LUISS_Study-Executive-summary.,Ref. Ares(2020)4713545 - 09/09/2020\n\nTHE EUR...
3,F541861,F541861-ACT_-_Perspectives_on_the_NCT_-_FINAL.,ACT PERSPECTIVES ON THE NEW COMPETITION TOOL \...
4,F549332,F549332-MCA_DSA_and_NCT_public_consultation_po...,The Malta Communications Authority’s ratio...


`df_text` is merged with the `dma` dataframe created in the [Data Description](#data-description) section using the unique Reference code of each document and respondent. Out of the 188 survey responses, only 85 have pdf submissions and some are duplicated. 

In [33]:
df_merged = dma.merge(df_text, how='left', on='Reference')
df_merged.head()

,Reference,Feedback date,Language,User type,First name,Surname,Scope,Organisation name,Transparency register number,Organisation size,...,Q201,Q202,Q203,Q204,Q205,Q206,submit,label_132,file_name,text
0,F550828,8.09.2020 23:57,English,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Most effective,Very effective,See our response above.,NaN,No.,Yes,0,Yes,NaN,NaN
1,F550827,8.09.2020 23:54,English,NGO (Non-governmental organisation),Juliane,von Reppert-Bismarck,NaN,Lie Detectors,094738529674-10,Small (< 50 employees),...,Most effective,Most effective,An additional regulatory framework imposing ob...,Lie_Detectors_-_Digital_Services_Act_-_New_Com...,Please see attached submission,Yes,1,Yes,F550827-Lie_Detectors_-_Digital_Services_Act_-...,"8 Sept, 2020 \n\nDigital Services Act and New ..."
2,F550826,8.09.2020 23:50,German,Business Association,Antje,Woltermann,NaN,Zentralverband Deutsches Kfz-Gewerbe e.V.,71649103246-10,Medium (< 250 employees),...,Sufficiently effective,Very effective,-,Positionspapier_Gleichberechtigter_Zugang_zum_...,-,Yes,1,Yes,F550826-Positionspapier_Gleichberechtigter_Zug...,Ref. Ares(2020)4723306 - 10/09/2020\n\nGemeins...
3,F550825,8.09.2020 23:45,English,Academic/Research Institution,DIANA,MONTENEGRO,NaN,Master student candidate Master in Competition...,NaN,Large (250 or more),...,Very effective,Very effective,all are adequated,PREPARATIVE_DOCUMENT_FOR_THE_EUROEPAN_COMMISIO...,The document I prepared was more than 1MB.,Yes,1,Yes,F550825-PREPARATIVE_DOCUMENT_FOR_THE_EUROEPAN_...,Ref. Ares(2020)4723305 - 10/09/2020\n\nTHE LEG...
4,F550824,8.09.2020 23:33,English,Business Association,Kamila,Sotomska,NaN,Zwi?zek Przedsi?biorców i Pracodawców,868073924175-77,Small (< 50 employees),...,Not applicable /No relevant experience or know...,Not applicable /No relevant experience or know...,NaN,NaN,NaN,Yes,0,Yes,NaN,NaN


For the next step, we only use responses with both PDF submission and label.

In [34]:
df_merged = df_merged[df_merged['submit']==1]
len(df_merged)

93

### A.2 Cleaning and Pre-Processing

After parsing PDFs, we need to further process the raw text to ensure that most of the information we feed into our model/s is relevant to the task at hand. For instance, stopwords like "the", "this", "and" will not give us any indication of whether a stakeholder agrees with a law, so we can remove these words. 

The pre-processing steps that we apply for this tutorial are the following: 

1. Removal of stopwords, punctuations, and numeric characters
2. Stemming and lemmatization
3. Coreference resolution
4. Language detection

This is what the raw text looks like before processing.

In [35]:
df_merged.text[1]

'8 Sept, 2020 \n\nDigital Services Act and New Competition Tool  \n\nLie Detectors response to public consultations \n\nLie  Detectors,  an  award-winning  journalist-led  media  literacy  campaign  in  Europe,  welcomes  the \nability to respond to the Commission consultation on the Digital Services Act and the New Competition \nTool.  \n\nIn relation to the Digital Services Act consultation, Lie Detectors supports the introduction of Ex Ante \nRegulation for large online platforms with significant network effects acting as gatekeepers, and of a \nNew Competition Tool.  \n\nThe  instinct  of  regulators  and  policymakers  to  hold  large  platforms  like  Facebook  and  Google  to \naccount for their role in the proliferation of online disinformation and in the undermining of quality \njournalism is right. With the necessary political will, solutions exist that will help rein in the epidemic \nof  disinformation  that  is  sweeping  away  trust  in  established  facts,  in  scientifi

#### Removal of stopwords, punctuations, numeric characters

In [36]:
def preprocess_corpus(texts):
    eng_stopwords = set(stopwords.words("english"))
    def remove_stops_digits(tokens):
        token_list =  [token.lower() for token in tokens if token not in eng_stopwords and token not in punctuation and token.isdigit() == False]
        processed_text = ' '.join(token_list)
        return processed_text
    return [remove_stops_digits(word_tokenize(text)) for text in texts]

In [37]:
df_merged['text_clean'] = preprocess_corpus(df_merged['text'])

#### Stemming and lemmatization

In [38]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [39]:
def stem_lemmatize(text):
    stemmed = [stemmer.stem(token) for token in word_tokenize(text)]
    lemmatized = [lemmatizer.lemmatize(token) for token in stemmed]
    processed_text = ' '.join(lemmatized)
    return processed_text

df_merged['text_clean'] = [stem_lemmatize(text) for text in df_merged['text_clean']]

#### Coreference resolution

In [ ]:
# installing neuralcoref from source
!git clone https://github.com/huggingface/neuralcoref.git
!cd neuralcoref
!pip install -r requirements.txt
!pip install -e .

In [24]:
import spacy
import neuralcoref

nlp = spacy.load('en_core_web_sm') 
neuralcoref.add_to_pipe(nlp)

In [ ]:
def coref_res(texts):
    doc = nlp(texts)
    clean = doc._.coref_resolved
    return clean

df_merged['text_clean'] = [coref_res(text) for text in df_merged['text_clean']]

#### Language detection

Subset dataframe to retain only english text

In [40]:
for index, row in df_merged.iterrows():
    df_merged.at[index, 'lang'] = detect(df_merged.at[index, 'text_clean'])

df_merged.lang.value_counts()

en    83
de     7
fr     2
es     1
Name: lang, dtype: int64

In [41]:
df_merged = df_merged[df_merged['lang']=="en"]

#### Final dataframe

In [43]:
df_final = df_merged[['Reference', 'Feedback date', 'User type', 'Scope', 
                    'Organisation name', 'Transparency register number', 'Organisation size',
                    'label_132', 'submit', 'file_name', 'lang', 'text', 'text_clean']]

In [ ]:
df_final.reset_index(drop=True).to_json(r"../data/df_final_document.json")

The pre-processed text should look like this.

In [45]:
df_final.text_clean[1]

'sept digit servic act new competit tool lie detector respons public consult lie detector award-win journalist-l medium literaci campaign europ welcom abil respond commiss consult digit servic act new competit tool in relat digit servic act consult lie detector support introduct ex ant regul larg onlin platform signific network effect act gatekeep new competit tool the instinct regul policymak hold larg platform like facebook googl account role prolifer onlin disinform undermin qualiti journal right with necessari polit solut exist help rein epidem disinform sweep away trust establish fact scientif method democrat institut design protect u the basi solut lie take disinform sourc take busi model larg platform facebook googl stoke outrag revenu engag “ monetis lie ” european commiss call follow money appli exist new antitrust principl fundament avenu secur european democraci other approach proven incap dent outrag economi corros effect democraci fact-check initi long darl conflict-shi re

In [13]:
df = pd.read_json(r"./data/df_final_document.json")

## B. Text Representation
In this section of the tutorial, we will demonstrate how to **convert raw text into numerical form** that can be readily fed into different machine learning and deep learning algorithms. There are two main ways to do this - (1) basic vectorization and (2) distributed representations. 

### B.1 Basic Vectorization
Basic vectorization techniques map each word of the corpus vocabulary (V) to a numeric value and represents each document as a V-dimensional vector. These methods are simple and straightforward and can be used to construct ML-based text and document classifiers with interpretable features. The most common methods are one-hot encoding and term frequency–inverse document frequency (TF-IDF). 

#### TF-IDF

Our TF-IDF matrix has 143 documents with 12,077 features or words.

In [69]:
vectorizer = TfidfVectorizer()
dfm = vectorizer.fit_transform(df['text_clean'])
dfm.shape

(143, 12077)

### B.2 Distibuted Representations
Distributed representations or **embeddings** are dense, low-dimensional vectors which capture context and distributional similarities between words. These methods address some fundamental limitations of basic vectorization techniques. First, distributed representations are more computationally efficient since they don't retain the shape of the entire corpus vocabulary. Second, context and similarities between words are accounted for, unlike basic vectorizations where words are treated as atomic units. Finally, they provide a solution to the *out of vocabulary* problem, where a model is unable to represent a word that was not used in the training data. One of the biggest advantage of embeddings is that they are able generalization to unseen text.

Word or document embeddings that may be pre-trained based on a big corpus (e.g. Word2Vec or GloVe) or trained based on your own set of documents (using CBOW or SkipGram).
Below is an illustration of how to construct TF-IDF weighted word embeddings for our set of DMA consultation documents using pre-trained GloVe embeddings.

First, load the GloVe embeddings which can be download from the NLP Stanford <a href = "https://nlp.stanford.edu/projects/glove/">website</a>. We will use the 100-dimensional GloVe embeddings and save it into a matrix.

In [57]:
dims = 100

z = ZipFile("data/glove.6B.zip") # glove zip file saved in data folder
f = z.open(f'glove.6B.{dims}d.txt')

embed_matrix = pd.read_table(
    f, sep = " ", index_col = 0, 
    header = None, quoting = csv.QUOTE_NONE
)

The GloVe embeddings has a 400,000-word vocabulary, with each word represented by a vector of size 100. 

In [60]:
embed_matrix.shape

(400000, 100)

In [61]:
embed_matrix

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
the,-0.038194,-0.244870,0.728120,-0.399610,0.083172,0.043953,-0.391410,0.334400,-0.57545,0.087459,...,0.016215,-0.017099,-0.389840,0.87424,-0.725690,-0.510580,-0.520280,-0.145900,0.82780,0.270620
",",-0.107670,0.110530,0.598120,-0.543610,0.673960,0.106630,0.038867,0.354810,0.06351,-0.094189,...,0.349510,-0.722600,0.375490,0.44410,-0.990590,0.612140,-0.351110,-0.831550,0.45293,0.082577
.,-0.339790,0.209410,0.463480,-0.647920,-0.383770,0.038034,0.171270,0.159780,0.46619,-0.019169,...,-0.063351,-0.674120,-0.068895,0.53604,-0.877730,0.318020,-0.392420,-0.233940,0.47298,-0.028803
of,-0.152900,-0.242790,0.898370,0.169960,0.535160,0.487840,-0.588260,-0.179820,-1.35810,0.425410,...,0.187120,-0.018488,-0.267570,0.72700,-0.593630,-0.348390,-0.560940,-0.591000,1.00390,0.206640
to,-0.189700,0.050024,0.190840,-0.049184,-0.089737,0.210060,-0.549520,0.098377,-0.20135,0.342410,...,-0.131340,0.058617,-0.318690,-0.61419,-0.623930,-0.415480,-0.038175,-0.398040,0.47647,-0.159830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chanty,-0.155770,-0.049188,-0.064377,0.223600,-0.201460,-0.038963,0.129710,-0.294510,0.00359,-0.098377,...,0.093324,0.094486,-0.023469,-0.48099,0.623320,0.024318,-0.275870,0.075044,-0.56380,0.145010
kronik,-0.094426,0.147250,-0.157390,0.071966,-0.298450,0.039432,0.021870,0.008041,-0.18682,-0.311010,...,-0.305450,-0.011082,0.118550,-0.11312,0.339510,-0.224490,0.257430,0.631430,-0.20090,-0.105420
rolonda,0.360880,-0.169190,-0.327040,0.098332,-0.429700,-0.188740,0.455560,0.285290,0.30340,-0.366830,...,-0.044082,0.140030,0.300070,-0.12731,-0.143040,-0.069396,0.281600,0.271390,-0.29188,0.161090


We find the words in our corpus that are present in the GloVe embedding matrix. Results below show that there are 7,462 in common with the GloVe embedding matrix. We then get the indices of these common words.

In [64]:
common_features = set(embed_matrix.index) & set(vectorizer.get_feature_names_out())
len(common_features)

7462

In [48]:
vocab_ids = [vectorizer.vocabulary_[x] for x in common_features]
vocab_ids[1:10]

Using common feature only, we multiply our 143 x 7462 TF-IDF matrix by the 7462 x 100 embedding matrix to get the document embedding matrix for all consultation documents. 

In [70]:
doc_matrix = dfm[:,vocab_ids].dot(embed_matrix.loc[common_features,])
doc_matrix.shape

/var/folders/rd/n9w0gpv53y72x5k9wk3hp63w0000gn/T/ipykernel_32263/3754221714.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  doc_matrix = dfm[:,vocab_ids].dot(embed_matrix.loc[common_features,])


(143, 100)

## C. Model Training (with hyperparameter tuning)
In this section of the tutorial, we compare 3 different types text classification models.

In [88]:
le = LabelEncoder()
df['label.132'] = le.fit_transform(df['label_132'])
df['label.134'] = le.fit_transform(df['label_134'])

### C.1 Baseline Models
For our baseline models, we will use traditional ML classifiers - logistic regression and gradient boosting - with the document embedding matrix as features.

##### Logistic Regression

In [89]:
# split data into train, test, validation sets
train_text, test_text, train_labels, test_labels = train_test_split(doc_matrix, df['label.132'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['label.132'])

In [90]:
clf = LogisticRegression(random_state=0).fit(train_text, train_labels)

y_pred = clf.predict(test_text)

accuracy = accuracy_score(test_labels, y_pred) *100.0
precision = precision_score(test_labels, y_pred, average='binary')
recall = recall_score(test_labels, y_pred, average='binary')
f_score = 2 * (precision * recall) / (precision + recall)

print(f' Accuracy: {accuracy:.2f} \n Precision: {precision:.3f} \n Recall: {recall:.3f} \n F1: {f_score:.3f}')

 Accuracy: 67.44 
 Precision: 0.700 
 Recall: 0.808 
 F1: 0.750


/Users/janinedevera/opt/miniconda3/envs/watermelon/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### Gradient Boosting

In [91]:
bst = XGBClassifier(n_estimators=1000, max_depth=1000, learning_rate=0.1, objective='binary:logistic')

bst.fit(train_text, train_labels)

print(bst)

y_pred = bst.predict(test_text)

accuracy = accuracy_score(test_labels, y_pred) * 100.0
precision = precision_score(test_labels, y_pred, average='binary')
recall = recall_score(test_labels, y_pred, average='binary')
f_score = 2 * (precision * recall) / (precision + recall)

print(f' Accuracy: {accuracy:.2f} \n Precision: {precision:.3f} \n Recall: {recall:.3f} \n F1: {f_score:.3f}')

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=1000, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)
 Accuracy: 62.79 
 Precision: 0.667 
 Recall: 0.769 
 F1: 0.714


### C.2 Long Short-Term Memory Network

After splitting the data into training and test set, build the corpus vocabulary by tokenizing all texts and assigning each word to a unique index.

In [10]:
tokenizer = get_tokenizer("basic_english")

def tokenize(datasets):
    for dataset in datasets:
        for text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(tokenize([train_text, test_text]), min_freq=1, specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

In [11]:
# example
tokens = tokenizer("This is an example.")
index = vocab(tokens)
index

[7332, 1493, 719, 6370, 1]

Once the vocabulary is built, we create batches of text sequences and map the tokens to indices. We also pad the sequence of words so all are of the same length. This returns a tensor of the sequence length and batch size. 

In [12]:
target_classes = ["0", "1"]
max_words = 100

def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X] # map tokes to index using vocab
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] # pad sequences

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=100, collate_fn=vectorize_batch, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=100, collate_fn=vectorize_batch)

In [14]:
for X, Y in train_loader:
    print(X.shape, Y.shape)
    break

torch.Size([100, 100]) torch.Size([100])


Create LSTM classifier

In [15]:
# define hyperparameters
embed_len = 100
hidden_dim = 60
n_layers = 2

class LSTMClassifier(nn.Module):
    def __init__(self):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.embed_len = embed_len
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        hidden, carry = torch.randn(n_layers, len(X_batch), hidden_dim), torch.randn(n_layers, len(X_batch), hidden_dim)
        output, (hidden, carry) = self.lstm(embeddings, (hidden, carry))
        return self.linear(output[:,-1])

    def init_hidden(self):
      return (
               torch.zeros(n_layers, 1, self.hidden_dim, device=device),
               torch.zeros(n_layers, 1, self.hidden_dim, device=device)
            )

In [16]:
lstm_classifier = LSTMClassifier()
lstm_classifier

LSTMClassifier(
  (embedding_layer): Embedding(14499, 100)
  (lstm): LSTM(100, 60, num_layers=2, batch_first=True)
  (linear): Linear(in_features=60, out_features=2, bias=True)
)

In [17]:
for layer in lstm_classifier.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print()

Layer : Embedding(14499, 100)
Parameters : 
torch.Size([14499, 100])

Layer : LSTM(100, 60, num_layers=2, batch_first=True)
Parameters : 
torch.Size([240, 100])
torch.Size([240, 60])
torch.Size([240])
torch.Size([240])
torch.Size([240, 60])
torch.Size([240, 60])
torch.Size([240])
torch.Size([240])

Layer : Linear(in_features=60, out_features=2, bias=True)
Parameters : 
torch.Size([2, 60])
torch.Size([2])



In [18]:
out = lstm_classifier(torch.randint(0, len(vocab), (1024, max_words)))
out.shape

torch.Size([1024, 2])

In [19]:
def evaluate(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))

def train(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y) 
            losses.append(loss.item())

            ## back propagation
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step() 

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        evaluate(model, loss_fn, val_loader)

In [20]:
from torch.optim import Adam

epochs = 20
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
lstm_classifier = LSTMClassifier()
optimizer = Adam(lstm_classifier.parameters(), lr=learning_rate)

train(lstm_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

100%|██████████| 1/1 [00:00<00:00,  1.81it/s]


Train Loss : 0.697
Valid Loss : 0.693
Valid Acc  : 0.512


100%|██████████| 1/1 [00:00<00:00,  3.76it/s]


Train Loss : 0.690
Valid Loss : 0.690
Valid Acc  : 0.535


100%|██████████| 1/1 [00:00<00:00,  3.89it/s]


...

Train Loss : 0.487
Valid Loss : 0.660
Valid Acc  : 0.628


100%|██████████| 1/1 [00:00<00:00,  3.98it/s]


Train Loss : 0.463
Valid Loss : 0.665
Valid Acc  : 0.628


100%|██████████| 1/1 [00:00<00:00,  4.00it/s]

Train Loss : 0.437
Valid Loss : 0.671
Valid Acc  : 0.628


### C.3 Transfer Learning with BERT and Other Variants

**Bidirectional Encoder Representations from Transformers** or **BERT** follows the Transformer architecture but it only uses the encoder part. Each transformer encoder consists of two sub-layers: self-attention and feed-forward. The key feature of BERT is its bidirectionality as it learns information from both left and right side of a token’s context. This allows for parallel processing, store the position of the input, and make lookup easy. 


BERT is trained from BookCorpus (800M words) and English Wikipedia (2.5B words). In a nutshell, BERT reads the input sequence and generates meaningful text representations, which it feeds into the encoder. This can then be augmented with additional neural network layers to fit a classification task. BERT has three embedding layers: token embedding layer, segment embedding layer, and position embedding layer. The element-wise sum of these three layers gives the final input representation.

<p align="center">
    <img src="./img/bert_embeddings.png" width="250" height="320">
</p>

For NLP related task, the best practice is to use pre-trained models first, and then fine-tune them. This is called **transfer learning**, a technique where a deep learning model trained from a very large dataset is used as “off-the-shelf” to perform similar tasks on another dataset. There are 3 different fine-tuning techniques:
> 1. Train the entire architecture which updates all pre-trained weights based on the new dataset
> 2. Train the pretrained model partially and freeze the weights of the initial layer and retrain only on the higher levels
> 3. Freeze entire architecture and attach a few NN layers to train the new model.

In this tutorial, we will implement transfer model using a pre-trained BERT model. After collecting, preparing, and pre-processing the unstructured documents, we split the dataset into training, validation, and testing. We then implement transfer learning using the bert-base-uncased model with 12 encoder transformer layers trained on lower-case English texts. We use the wordpiece embeddings from BERT as inputs to our text classification task.

<p align="center">
    <img src="./img/bert_pipeline.png" width="700" height="320">
<p>

In [56]:
# split data into train, test, validation sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text_clean'], df['label.132'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['label.132'])

val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

##### Import BERT Model and BERT Tokenizer

In [15]:
# bert-base-uncased
bert = AutoModel.from_pretrained('bert-base-uncased', return_dict=False)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', return_dict=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# legal-bert-base-uncased
bert = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased", return_dict=False)
tokenizer = BertTokenizerFast.from_pretrained("nlpaueb/legal-bert-base-uncased", return_dict=False)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##### Tokenize the Sentences

In [18]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

c:\Users\admin\anaconda3\envs\watermelon\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [21]:
# convert lists to tensors
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [22]:
batch_size = 2
num_workers = 2

# dataLoader for train set
train_data = TensorDataset(train_seq, train_mask, train_y)
train_dataloader = DataLoader(train_data, num_workers=num_workers, shuffle=True, batch_size=batch_size)

# dataLoader for validation set
val_data = TensorDataset(val_seq, val_mask, val_y)
val_dataloader = DataLoader(val_data, num_workers=num_workers, shuffle=True, batch_size=batch_size)

##### BERT Model Architecture

In [23]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)
      self.fc2 = nn.Linear(512,2)
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)
      x = self.softmax(x)

      return x

In [24]:
# method to freeze all the parameters if freeze = T
def set_parameter_requires_grad(model, freeze):
    if freeze:
        for param in model.parameters():
            param.requires_grad = False

In [25]:
# freeze all parameters
set_parameter_requires_grad(model=bert, freeze=True)

# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [26]:
# optimizer from hugging face transformers
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-5) # learning rate

# loss function
criterion  = nn.NLLLoss() 

# no. of training epochs
epochs = 20

c:\Users\admin\anaconda3\envs\watermelon\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


##### Fine-tuning BERT

In [27]:
# function to train the model
def train(model, dataloader, criterion, optimizer):
  
  model.train()
  total_loss, total_accuracy = 0, 0
  running_total_correct = 0.0
  total_preds=[]
  
  for i, inputs in enumerate((dataloader)):
    
    # push to gpu
    inputs = [r.to(device) for r in inputs]
    sent_id, mask, labels = inputs

    # zero the parameter gradients
    model.zero_grad()        

    # forward + backward + optimize 
    preds = model(sent_id, mask)
    loss = criterion(preds, labels)
    total_loss = total_loss + loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) #prevent exploding gradient problem
    optimizer.step()
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # epoch loss and accuracy
  epoch_loss = total_loss / len(dataloader)
  # reshape from (no. of batches, size of batch, no. of classes) to (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  print(f"Train Loss: {epoch_loss:.2f}")

  return epoch_loss, total_preds


In [28]:
# function for evaluating the model
def evaluate(model, dataloader, criterion):
  
  model.eval()
  total_loss, total_accuracy = 0, 0
  total_preds = []

  for i, inputs in enumerate((dataloader)):
    
    # push to gpu
    inputs = [t.to(device) for t in inputs]
    sent_id, mask, labels = inputs

    with torch.no_grad():
      preds = model(sent_id, mask)
      loss = criterion(preds,labels)
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
      total_preds.append(preds)

  # epoch loss and accuracy
  epoch_loss = total_loss / len(dataloader) 
  total_preds  = np.concatenate(total_preds, axis=0)
  print(f"Validation Loss: {epoch_loss:.2f}")

  return epoch_loss, total_preds

In [29]:
# set initial loss to infinite
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    train_loss, _ = train(model, train_dataloader, criterion, optimizer)
    valid_loss, _ = evaluate(model, val_dataloader, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    


 Epoch 1 / 20
Train Loss: 0.67
Validation Loss: 0.70

 Epoch 2 / 20
Train Loss: 0.67
Validation Loss: 0.69

 Epoch 3 / 20
Train Loss: 0.68
Validation Loss: 0.69

 Epoch 4 / 20
Train Loss: 0.67
Validation Loss: 0.69

 Epoch 5 / 20
Train Loss: 0.67
Validation Loss: 0.67

 Epoch 6 / 20
Train Loss: 0.69
Validation Loss: 0.66

 Epoch 7 / 20
Train Loss: 0.67
Validation Loss: 0.69

 Epoch 8 / 20
Train Loss: 0.66
Validation Loss: 0.64

 Epoch 9 / 20
Train Loss: 0.66
Validation Loss: 0.67

 Epoch 10 / 20
Train Loss: 0.66
Validation Loss: 0.66

 Epoch 11 / 20
Train Loss: 0.67
Validation Loss: 0.65

 Epoch 12 / 20
Train Loss: 0.67
Validation Loss: 0.65

 Epoch 13 / 20
Train Loss: 0.68
Validation Loss: 0.70

 Epoch 14 / 20
Train Loss: 0.67
Validation Loss: 0.72

 Epoch 15 / 20
Train Loss: 0.67
Validation Loss: 0.69

 Epoch 16 / 20
Train Loss: 0.66
Validation Loss: 0.67

 Epoch 17 / 20
Train Loss: 0.68
Validation Loss: 0.67

 Epoch 18 / 20
Train Loss: 0.66
Validation Loss: 0.71

 Epoch 19 / 20
Tra

In [59]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [60]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [61]:
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.59      1.00      0.74        13

    accuracy                           0.59        22
   macro avg       0.30      0.50      0.37        22
weighted avg       0.35      0.59      0.44        22



c:\Users\admin\anaconda3\envs\watermelon\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\anaconda3\envs\watermelon\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\anaconda3\envs\watermelon\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## D. Model Evaluation

The base line model even if htea are less complex then the BERT alghorithem give a good starting point to evaluate how good the final model is. The accuracy of the logistic regression is nearly 68 %. The result is quite imporessive considering the semplicity of the model. 

Implementing a LSTM layer and using a pretrained algorithem, the hope is to out perform the logistic regression. 
It is expected that the results in the final model would be better since there are many improvents. Moreover, there is an extra fully connected layerthe in the BERT model that is trained on the specifc data for the task. This should brin´g a significance increase in the overall accuracy. 

The last step is to fine tuning the model to select the best hyper parameters to achive the best performace. The new accuracy after all the stapes is ....

<a name="results-and-discussion"></a>
# Results & Discussion

In this section, describe and contextualize the results shown in the tutorial. Briefly describe the performance metrics and cross validation techniques used. 

Finally, include a discussion on the limitations and important takeaways from the exercise.

## Limitations
*   The tutorial is focused on education and learning. Explain all the simplifications you have made compared to applying a similar approach in the real world (for instance, if you have reduced your training data and performance).
*   ML algorithms and datasets can reinforce or reflect unfair biases. Reflect on the potential biases in the dataset and/or analysis presented in your tutorial, including its potential societal impact, and discuss how readers might go about addressing this challenge. 

## Next Steps
*   What do you recommend would be the next steps for your readers after finishing your tutorial?
*   Discuss other potential policy- and government-related applications for the method or tool discussed in the tutorial.
*   List anything else that you would want the reader to take away as they move on from the tutorial.

<a name="references"></a>
# References

Include all references used. 

For example, in this template:

*   EarthCube Notebook Template: https://github.com/earthcube/NotebookTemplates
*   Earth Engine Community Tutorials Style Guide: https://developers.google.com/earth-engine/tutorials/community/styleguide#colab
*   Google Cloud Community Tutorial Style Guide: https://cloud.google.com/community/tutorials/styleguide
*   Rule A, Birmingham A, Zuniga C, Altintas I, Huang S-C, Knight R, et al. (2019) Ten simple rules for writing and sharing computational analyses in Jupyter Notebooks. PLoS Comput Biol 15(7): e1007007. https://doi.org/10.1371/journal.pcbi.1007007




## Acknowledgement

These guidelines are heavily based on the Climate Change AI template for the for the tutorials track at the [NeurIPS 2021 Workshop on Tackling Climate Change with Machine Learning](https://www.climatechange.ai/events/neurips2021). 